In [21]:
# Author Kees van den Tempel, AI-labs BV, Tricht (kees@ai-labs.nl)
# Versie 1.1  -- Update voor nieuwe Api van OpenAI
# 4-1-2024

# This script reads 15630 questions and answers (QA) of the products and services of the dutch government and makes
# it semantic searchable by using the Chroma vector database

# When the QA-dataset is read into the database, it is embedded, using the standard Chroma emmbedding algorithm
# When all the data is in the Chroma database it is possible to serach for identical questions and for the best
# answers

# Kijk ook op https://thenewstack.io/tutorial-use-chroma-and-openai-to-build-a-custom-qa-bot/

%pip install --upgrade pip
%pip install ipywidgets
%pip install typing-extensions==4.7.1
%pip install --upgrade datetime
%pip install pandas --upgrade
%pip install numpy --upgrade
%pip install --upgrade matplotlib
%pip install --upgrade scikit-learn

%pip install -Uq chromadb numpy datasets
%pip install openai --upgrade


Note: you may need to restart the kernel to use updated packages.
  Using cached ipywidgets-8.1.1-py3-none-any.whl.metadata (2.4 kB)
Using cached ipywidgets-8.1.1-py3-none-any.whl (139 kB)
Note: you may need to restart the kernel to use updated packages.
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi 0.108.0 requires typing-extensions>=4.8.0, but you have typing-extensions 4.7.1 which is incompatible.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached numpy-1.26.3-cp311-cp311-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.3-cp311-cp311-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Rolling back uninstall of numpy
  Moving to c:\python311\lib\site-packages\numpy-1.25.2.dist-info\
   from C:\Python311\Lib\site-packages\~umpy-1.25.2.dist-info
  Moving to c:\python311\lib\site-packages\numpy\
   from C:\Python311\Lib\site-packages\~umpy
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 2] Het systeem kan het opgegeven bestand niet vinden: 'c:\\Python311\\Scripts\\f2py.exe' -> 'c:\\Python311\\Scripts\\f2py.exe.deleteme'



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 2] Het systeem kan het opgegeven bestand niet vinden: 'c:\\Python311\\Scripts\\f2py.exe' -> 'c:\\Python311\\Scripts\\f2py.exe.deleteme'



Note: you may need to restart the kernel to use updated packages.


In [15]:
import numpy as np
import pandas as pd
import openai
import time
import json

# Read the file with the most-asked Q&A of the products and service of all Dutch governments
QAdata = pd.read_csv("data/NL_ProdenDienstenQA.csv", decimal=".", sep="#")
QAdata['ID'] = QAdata.index

print(QAdata.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15618 entries, 0 to 15617
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Vraag                15618 non-null  object
 1   Antwoord             15618 non-null  object
 2   Nummer               15618 non-null  int64 
 3   Instantie            15618 non-null  object
 4   UniformeProductnaam  15618 non-null  object
 5   Grondslaglabel       15618 non-null  object
 6   Overheid             15618 non-null  object
 7   SoortProduct         15618 non-null  object
 8   Klant                15618 non-null  object
 9   Prompt2              15618 non-null  object
 10  ID                   15618 non-null  int64 
dtypes: int64(2), object(9)
memory usage: 1.3+ MB
None


In [16]:
QAdata.head(1)

,Vraag,Antwoord,Nummer,Instantie,UniformeProductnaam,Grondslaglabel,Overheid,SoortProduct,Klant,Prompt2,ID
0,Wat is een aanvullende beurs kwijtschelding?,Een aanvullende beurs kwijtschelding is een fi...,1,Dienst Uitvoering Onderwijs (DUO),aanvullende beurs kwijtschelding,Artikel 6.2 Wet studiefinanciering 2000,de Nederlandse Rijksoverheid,een aanvraag doen,burger,Een burger uit Nederland wil een aanvraag doen...,0


In [23]:
import typing_extensions
from importlib import reload
reload(typing_extensions)
import chromadb

# Delete a collection and all associated embeddings, documents, and metadata. ⚠️ This is destructive and not reversible
try:
    chroma_client = chromadb.PersistentClient(path="data/chroma")
    chroma_client.delete_collection(name="Dutch_QA_collection") 
except:
    print("Geen Dutch_QA_collection collection aanwezig")
    
collection = chroma_client.create_collection(name="Dutch_QA_collection")


Geen Dutch_QA_collection collection aanwezig


In [24]:
# Convert the list of product names into the desired format
QAMeta = [{"ProductID": product} for product in QAdata['ID']]

for index in range(0, len(QAMeta[:100]), 10):
    print(f"Index: {index}, Value: {QAMeta[index]}")
    
# Print the length of the metadatas list
print("Length of metadatas:", len(QAMeta))

Index: 0, Value: {'ProductID': 0}
Index: 10, Value: {'ProductID': 10}
Index: 20, Value: {'ProductID': 20}
Index: 30, Value: {'ProductID': 30}
Index: 40, Value: {'ProductID': 40}
Index: 50, Value: {'ProductID': 50}
Index: 60, Value: {'ProductID': 60}
Index: 70, Value: {'ProductID': 70}
Index: 80, Value: {'ProductID': 80}
Index: 90, Value: {'ProductID': 90}
Length of metadatas: 15618


In [26]:
#QAdata['Document'] = "VRAAG: " + QAdata['Vraag'] + "ANTWOORD: " + QAdata['Antwoord'] + "\n"

# Assume QAList, QAMeta, and QAIds are your original data lists
batch_size = 5460  # Set to the maximum allowed batch size
num_batches = len(QAList) // batch_size + (1 if len(QAList) % batch_size > 0 else 0)

for i in range(num_batches):
    start_index = i * batch_size
    end_index = start_index + batch_size
    batch_QAList = QAList[start_index:end_index]
    batch_QAMeta = QAMeta[start_index:end_index]
    batch_QAIds = QAIds[start_index:end_index]

    # Process each batch
    try:
        collection.add(
            documents=batch_QAList,
            metadatas=batch_QAMeta,
            ids=batch_QAIds
        )
    except Exception as e:
        print(f"Error processing batch {i}: {e}")


In [27]:
collection = chroma_client.get_collection(name="Dutch_QA_collection")
Q = "ik wil een omgevingsvergunning aanvragen?"

results = collection.query(
    query_texts=[Q],
    n_results=30
)

In [28]:
# Convert the results to a DataFrame
df = pd.DataFrame({
    'ID': results['ids'][0],
    'Distance': results['distances'][0],
    'Metadata': results['metadatas'][0],
    'Document': results['documents'][0]
})

df = df.drop_duplicates(subset=['Document', 'Distance'])
df['Metadata'] = df['Metadata'].apply(lambda x: x.get('ProductID') if isinstance(x, dict) else x)
df['Metadata'] = df['Metadata'].astype(int)

# Display the DataFrame
pd.set_option('display.max_colwidth', None)
print(df[['Metadata', 'Document', 'Distance']])

    Metadata                                                 Document  \
0       1512            Hoe kan ik een omgevingsvergunning aanvragen?   
8       5244     Hoe kan ik een omgevingsvergunning aanleg aanvragen?   
12      3522     Hoe kan ik een omgevingsvergunning uitweg aanvragen?   
20      5254  Hoe kan ik een omgevingsvergunning afwijking aanvragen?   
22     10718   Hoe kan ik een woningbemiddelingsvergunning aanvragen?   
23      5324     Hoe kan ik een omgevingsvergunning natuur aanvragen?   
25      5334     Hoe kan ik een omgevingsvergunning opslag aanvragen?   
26      1652         Hoe kan ik een ontgrondingsvergunning aanvragen?   

    Distance  
0   0.182463  
8   0.194487  
12  0.228119  
20  0.242482  
22  0.246985  
23  0.249920  
25  0.250155  
26  0.262419  


In [30]:
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

def get_completion(prompt, model="gpt-3.5-turbo"):

    #This function takes a prompt and uses the OpenAI API to generate a chat completion response based on the given prompt.

    #Parameters:
    #    prompt (str): The input text prompt provided by the user.
    #    model (str): The name or identifier of the language model to be used for chat completion.
    #                 Default value is "gpt-3.5-turbo", which is a variant of the GPT-3.5 model.

    #Returns:
    #    str or None: The chat completion response generated by the model, or None if the API call fails.

    #Note:
    #    Before using this function, you need to have a valid OpenAI API key to authorize the API calls.
    #    The API key should be set as `openai.api_key` before invoking this function.

    client = OpenAI(api_key=OPENAI_API_KEY)

    # openai.api_key = MyApiKey
    messages = [{"role": "user", "content": prompt}]
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0 
        )
        txtTmp = response.choices[0].message.content
        return txtTmp
    except openai.OpenAIError as e:
        print(f"OpenAI API call failed: {e}")
        return None

result = get_completion("Wat is de beste auto die Volvo ooit gemaakt heeft?")
print(result)


Het is moeilijk om te zeggen wat de beste auto is die Volvo ooit heeft gemaakt, omdat dit afhangt van persoonlijke voorkeuren en behoeften. Volvo staat bekend om het produceren van veilige en betrouwbare auto's, en er zijn verschillende modellen die als hoogtepunten worden beschouwd. Enkele van de meest geprezen Volvo-modellen zijn onder andere de Volvo XC90, een luxe SUV met geavanceerde veiligheidsvoorzieningen, en de Volvo 240, een iconische en duurzame sedan die decennialang populair was. Het is belangrijk om te overwegen welke functies en specificaties voor jou het belangrijkst zijn bij het bepalen van de beste Volvo-auto.


In [31]:
# Merge the dataframes
merged_df = pd.merge(QAdata, df, left_on='ID', right_on='Metadata', how='inner')

# Sorting the merged dataframe by Metadata
sorted_df = merged_df.sort_values(by='Metadata')


In [38]:
# Select the 'Document' column of the first three records
documents = sorted_df['Antwoord'].iloc[:3]

# Concatenate the three text values with a separator (e.g., two newlines)
Antwoorden = "\n\n".join(documents)

prompt = "Ik ben een gemeente en een burger stelt de volgende vraag: " + Q + "\n"
prompt = prompt + "Wij hebben de drie volgende antwoorden gevonden: \n###" + Antwoorden + "###\n"
prompt = prompt + "Kun je de vraag en de antwoorden tot één samenhangende tekst omschrijven? " 
prompt = prompt + "Kun je dit doen op taalniveau b1?"

print("VRAAG AAN CHATGPT:")
print(prompt)

print("\n\nANTWOORD VAN OPENAI:")
print(get_completion(prompt, model="gpt-3.5-turbo"))

VRAAG AAN CHATGPT:
Ik ben een gemeente en een burger stelt de volgende vraag: ik wil een omgevingsvergunning aanvragen?
Wij hebben de drie volgende antwoorden gevonden: 
###U kunt een omgevingsvergunning aanvragen bij de gemeente waarin de activiteit plaatsvindt. Dit kan meestal online via de website van de gemeente. U dient hiervoor een aanvraagformulier in te vullen en de benodigde documenten mee te sturen.

U kunt een ontgrondingsvergunning aanvragen bij de Nederlandse Rijksoverheid. Hiervoor dient u een aanvraagformulier in te vullen en de benodigde documenten en informatie te verstrekken. Het aanvraagformulier en verdere instructies zijn te vinden op de website van de Rijksoverheid.

U kunt een omgevingsvergunning uitweg aanvragen bij uw gemeente. Neem contact op met de gemeente en vraag naar de procedure en benodigde documenten.###
Kun je de vraag en de antwoorden tot één samenhangende tekst omschrijven? Kun je dit doen op taalniveau b1?


ANTWOORD VAN OPENAI:
Als gemeente kunt u